### Import dependencies

In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.metrics import cohen_kappa_score

import sqlite3
import fasttext

### Import data

In [61]:
r3 = pd.read_csv('Manual_Review/round 3/Crowdfunding+-+Manual+Label+101-150_December+29,+2020_08.52.csv', header=None).T.iloc[17:]
r4 = pd.read_csv('Manual_Review/round 4/Crowdfunding+-+Manual+Label+Round+4_January+7,+2021_18.13.csv', header=None).T.iloc[17:]
r5 = pd.read_csv('Manual_Review/round 5/Crowdfunding+-+Manual+Label+Round+5_January+11,+2021_08.36.csv', header=None).T.iloc[17:]
r6 = pd.read_csv('Manual_Review/round 6/Crowdfunding+-+Manual+Label+Round+6_January+15,+2021_09.35.csv', header=None).T.iloc[17:]

### Inspect data

In [62]:
print(r3.shape)
print(r4.shape)
print(r5.shape)
print(r6.shape)

(150, 5)
(150, 4)
(150, 4)
(150, 5)


In [63]:
r3.head()

,0,1,2,3,4
17,QID2,https://www.gofundme.com/f/please-help-karens-...,"Infections,Injuries and external causes,Mental...",NaN,"Infections,Mental and substance use disorders,..."
18,QID3,https://www.gofundme.com/f/please-help-karens-...,Infections,NaN,Infections
19,QID4_TEXT,https://www.gofundme.com/f/please-help-karens-...,"Allergy, unspecified, subsequent encounter-->f...",NaN,Symptoms of mental illness d/t underlying food...
20,QID5,https://www.gofundme.com/f/patrickwalter__base...,Neoplasms,NaN,Neoplasms
21,QID6,https://www.gofundme.com/f/patrickwalter__top ...,Neoplasms,NaN,Neoplasms


In [64]:
r4.head()

,0,1,2,3
17,QID2,https://www.gofundme.com/f/healthecheryl__base...,"Nervous system diseases,Other noncommunicable ...","Nervous system diseases,Other noncommunicable ..."
18,QID3,https://www.gofundme.com/f/healthecheryl__top ...,Other noncommunicable diseases,Other noncommunicable diseases
19,QID4_TEXT,https://www.gofundme.com/f/healthecheryl__NOTE...,sarcoidosis listed under non communicable dise...,MS and sarcoidosis\nmain is sarcoidosis
20,QID5,https://www.gofundme.com/f/22kcir7teo__base - ...,"Cardiovascular diseases,Genitourinary disorder...","Cardiovascular diseases,Genitourinary disorder..."
21,QID6,https://www.gofundme.com/f/22kcir7teo__top - P...,Genitourinary disorders,Infections


In [65]:
r5.head()

,0,1,2,3
17,QID2,https://www.gofundme.com/f/d90qtg__base - Hell...,Cardiovascular diseases,"Cardiovascular diseases,Nonspecific"
18,QID3,https://www.gofundme.com/f/d90qtg__top - Pleas...,Cardiovascular diseases,Cardiovascular diseases
19,QID4_TEXT,https://www.gofundme.com/f/d90qtg__NOTES - Ple...,NaN,"congestive heart failure, clots in my lungs an..."
20,QID5,https://www.gofundme.com/f/breatheeasyaimee__b...,Nonspecific,Nonspecific
21,QID6,https://www.gofundme.com/f/breatheeasyaimee__t...,Nonspecific,Nonspecific


In [66]:
r6.head()

,0,1,2,3,4
17,QID2,https://www.gofundme.com/f/nc89hu-battling-sta...,"Neoplasms,Respiratory diseases",NaN,Neoplasms
18,QID3,https://www.gofundme.com/f/nc89hu-battling-sta...,Neoplasms,NaN,Neoplasms
19,QID4_TEXT,https://www.gofundme.com/f/nc89hu-battling-sta...,acute pulmonary edema,NaN,NaN
20,QID5,https://www.gofundme.com/f/javier-montenegros-...,Genitourinary disorders,NaN,Genitourinary disorders
21,QID6,https://www.gofundme.com/f/javier-montenegros-...,Genitourinary disorders,NaN,Genitourinary disorders


### Notes from inspection

r3
- need to delete column 3, this was a blank survey response
- col2 is dennis

r4
- col2 is dennis

r5
- col2 is dennis

r6
- col2 is steven
- col3 is dennis's second half
- col4 is dennis's first half
- need to merge 3 and 4

### Clean data

In [67]:
r3.columns = ['qid','header','dennis','blank','steven']
r4.columns = ['qid', 'header', 'dennis', 'steven']
r5.columns = ['qid', 'header', 'dennis', 'steven']
r6.columns = ['qid', 'header', 'steven', 'dennis_2', 'dennis_1']

In [68]:
r6 = r6.reset_index()

In [69]:
r6.head(70)

,index,qid,header,steven,dennis_2,dennis_1
0,17,QID2,https://www.gofundme.com/f/nc89hu-battling-sta...,"Neoplasms,Respiratory diseases",NaN,Neoplasms
1,18,QID3,https://www.gofundme.com/f/nc89hu-battling-sta...,Neoplasms,NaN,Neoplasms
2,19,QID4_TEXT,https://www.gofundme.com/f/nc89hu-battling-sta...,acute pulmonary edema,NaN,NaN
3,20,QID5,https://www.gofundme.com/f/javier-montenegros-...,Genitourinary disorders,NaN,Genitourinary disorders
4,21,QID6,https://www.gofundme.com/f/javier-montenegros-...,Genitourinary disorders,NaN,Genitourinary disorders
...,...,...,...,...,...,...
65,82,QID67_TEXT,https://www.gofundme.com/f/lets-help-ariel-kic...,NaN,NaN,NaN
66,83,QID68,https://www.gofundme.com/f/daphnee-strong__bas...,"Neoplasms,Nonspecific",NaN,Neoplasms
67,84,QID69,https://www.gofundme.com/f/daphnee-strong__top...,Neoplasms,NaN,Neoplasms
68,85,QID70_TEXT,https://www.gofundme.com/f/daphnee-strong__NOT...,NaN,NaN,NaN


In [70]:
first = r6['dennis_1'].iloc[:69].tolist()
second = r6['dennis_2'].iloc[69:].tolist()
new_dennis = first + second
r6.loc[:,'dennis'] = new_dennis

In [71]:
r3 = r3[['header','dennis','steven']]
r4 = r4[['header','dennis','steven']]
r5 = r5[['header','dennis','steven']]
r6 = r6[['header','dennis','steven']]

In [72]:
r3.loc[:,'round'] = [3 for x in range(len(r3))]
r4.loc[:,'round'] = [4 for x in range(len(r4))]
r5.loc[:,'round'] = [5 for x in range(len(r5))]
r6.loc[:,'round'] = [6 for x in range(len(r6))]

In [73]:
#rbind dfs
m = pd.concat([r3, r4, r5, r6])

In [74]:
#extract url from header
m['url'] = [x.split(' - ')[0].split('__')[0] for x in m['header']]

In [75]:
#extract question category from header
m['qcat'] = ['base','top','NOTES'] * 200

In [76]:
m = m[['url','qcat','dennis','steven','round']]

In [77]:
#subset data

# all categories that are in text
base = m[m['qcat'] == 'base']

# top (aka primary) category
top = m[m['qcat'] == 'top']

# notes
notes = m[m['qcat'] == 'NOTES']

In [78]:
print(base.shape)
print(top.shape)
print(notes.shape)

(200, 5)
(200, 5)
(200, 5)


In [79]:
#convert data from wide to long
dennis_melt = pd.melt(base, id_vars=['url','qcat','round'], value_vars=['dennis'])
steven_melt = pd.melt(base, id_vars=['url','qcat','round'], value_vars=['steven'])

In [80]:
#merge melted data and rename columns
melt = pd.concat([dennis_melt, steven_melt])
melt = melt.rename(columns={'variable':'rater','value':'cat'})

In [81]:
#reset index
melt = melt.reset_index()
del melt['index']

In [82]:
def comma2lofd(df):
    '''convert cat column from comma delimited to list of dictionaries with 1 or 0 for each category'''
    
    disease_categories = [
        'Cardiovascular diseases',
        'Endocrine diseases',
        'Gastrointestinal diseases',
        'Genitourinary disorders',
        'Infections',
        'Injuries and external causes',
        'Mental and substance use disorders',
        'Musculoskeletal diseases',
        'Neoplasms',
        'Nervous system diseases',
        'Nonspecific',
        'Other noncommunicable diseases',
        'Pregnancy and childbirth',
        'Respiratory diseases'
    ]
    
    r = []
    
    for index,row in df.iterrows():
        lofd = []
        as_list = row['cat'].split(',')
        for disease in disease_categories:
            if disease in as_list:
                lofd.append({disease:1})
            else:
                lofd.append({disease:0})
        r.append(lofd)
    return r

In [83]:
melt['lofd'] = comma2lofd(melt)
del melt['cat']

In [84]:
#convert to long format
long = melt.explode('lofd')

In [85]:
#extract key, value from dictionary
long.insert(loc=2, column='cat', value=[list(x.items())[0][0] for x in long['lofd']])
long.insert(loc=3, column='count', value=[list(x.items())[0][1] for x in long['lofd']])

long = long[['url','rater','round','cat','count']]

In [86]:
#create pivot tables
steven_pivot = long[long['rater'] == 'steven'].pivot_table(index=['url'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)
dennis_pivot = long[long['rater'] == 'dennis'].pivot_table(index=['url'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)

In [87]:
#calculate counts and percentage by category
dennis_counts = dennis_pivot.describe().T
dennis_counts['N'] = [round(x) for x in dennis_counts['count'] * dennis_counts['mean']]
dennis_counts['PER'] = dennis_counts['N'] / dennis_counts['count']

steven_counts = steven_pivot.describe().T
steven_counts['N'] = [round(x) for x in steven_counts['count'] * steven_counts['mean']]
steven_counts['PER'] = steven_counts['N'] / steven_counts['count']

In [88]:
cat = []
cohens_kappa = []
for col in dennis_pivot.columns:
    cat.append(col)
    cohens_kappa.append(cohen_kappa_score(dennis_pivot[col], steven_pivot[col]))
kappa_df = pd.DataFrame({'category':cat, 
              'cohens kappa':cohens_kappa,
              'dennis count':dennis_counts['N'],
              'steven count':steven_counts['N']}).sort_values('cohens kappa', ascending=False)

In [89]:
kappa_df

,category,cohens kappa,dennis count,steven count
cat,,,,
Neoplasms,Neoplasms,0.979169,81,79
Infections,Infections,0.927954,16,14
Musculoskeletal diseases,Musculoskeletal diseases,0.877900,18,18
Genitourinary disorders,Genitourinary disorders,0.871589,19,15
Endocrine diseases,Endocrine diseases,0.861624,12,11
Mental and substance use disorders,Mental and substance use disorders,0.825175,16,15
Injuries and external causes,Injuries and external causes,0.811083,32,23
Respiratory diseases,Respiratory diseases,0.807384,19,15
Cardiovascular diseases,Cardiovascular diseases,0.781399,31,28


In [90]:
kappa_df['cohens kappa'].mean()

0.7553637283841376

### Calculate Cohen's Kappa (top disease categories)

In [91]:
top = top.fillna('Nonspecific')
cohen_kappa_score(top['dennis'],top['steven'])

0.8202665013944841

# Resolving discrepancies

### Disagreement in primary category

In [92]:
def GetTopAgreement(df):    
    top_agreement = []
    for index,row in df.iterrows():
        if row['dennis'] == row['steven']:
            top_agreement.append(1)
        else:
            top_agreement.append(0)
    return top_agreement

In [93]:
top.loc[:,'agreement'] = GetTopAgreement(top)

In [94]:
top[top['agreement'] == 0]

,url,qcat,dennis,steven,round,agreement
36,https://www.gofundme.com/f/help-me-see-your-world,top,Other noncommunicable diseases,Nervous system diseases,3,0
48,https://www.gofundme.com/f/nbpzd4,top,Gastrointestinal diseases,Respiratory diseases,3,0
72,https://www.gofundme.com/f/i2hr7g,top,Genitourinary disorders,Cardiovascular diseases,3,0
150,https://www.gofundme.com/f/meals-for-healthcar...,top,Respiratory diseases,Nonspecific,3,0
21,https://www.gofundme.com/f/22kcir7teo,top,Genitourinary disorders,Infections,4,0
84,https://www.gofundme.com/f/bre039s-transplant,top,Genitourinary disorders,Musculoskeletal diseases,4,0
111,https://www.gofundme.com/f/ceiling-hoist-lift-...,top,Injuries and external causes,Other noncommunicable diseases,4,0
117,https://www.gofundme.com/f/support-for-twins-c...,top,Pregnancy and childbirth,Other noncommunicable diseases,4,0
120,https://www.gofundme.com/f/the-jon-neal-fund,top,Mental and substance use disorders,Neoplasms,4,0
153,https://www.gofundme.com/f/q2nr88,top,Injuries and external causes,Infections,4,0


### Disagreement in any category

In [95]:
def GetAnyDisagreement(d, s):
    r = []
    for i in range(len(d)):
        if d.iloc[i,:].tolist() == s.iloc[i,:].tolist():
            r.append(1)
        else:
            r.append(0)
    return r

In [96]:
dennis_pivot.loc[:,'agreement'] = GetAnyDisagreement(dennis_pivot, steven_pivot)

In [97]:
dennis_pivot['agreement'].value_counts()

1    133
0     67
Name: agreement, dtype: int64

### Make survey to review

In [98]:
def GetURLs(d, top):
    
    #define results containers
    any_and_top = []
    any_only = []
    
    #subset data
    d = d[d['agreement'] == 0]
    top = top[top['agreement'] == 0]
    
    #loop through disagreements
    for index, row in d.iterrows():
        #check if url is also in top disagreements
        if top['url'].str.contains(row['url']).any():
            any_and_top.append(row['url'])
        else:
            any_only.append(row['url'])
    
    return any_and_top, any_only

In [99]:
any_and_top, any_only = GetURLs(dennis_pivot.reset_index(), top)

In [100]:
top_only = top[top['agreement'] == 0]['url'][~top[top['agreement'] == 0]['url'].isin(any_and_top)].tolist()

### Import campaign data to populate questions

In [101]:
conn = sqlite3.connect("gfm.db")
feed = pd.read_sql_query("SELECT url, fund_description FROM feed_tb", conn)

In [102]:
feed = feed.drop_duplicates('url')
feed['fund_description'] = [x.replace('\n','') for x in feed['fund_description']]
feed['nchar']= [len(x) for x in feed['fund_description']]
feed = feed[feed['nchar'] >= 100]

In [103]:
def GetLanguage(df):
    path_to_pretrained_model = 'fasttext_models/lid.176.bin'
    fmodel = fasttext.load_model(path_to_pretrained_model)
    r = fmodel.predict(feed['fund_description'].to_list())
    labels = r[0]
    labels = [i[0].split('__')[2] for i in labels]
    scores = r[1]
    scores = [i[0] for i in scores]
    return labels, scores

In [104]:
feed['language'], feed['lang_score'] = GetLanguage(feed)

In [105]:
feed = feed[feed['language'] == 'en']

In [106]:
len(feed)

89676

### Check that urls are correct

In [107]:
r = []
for url in top_only:
    r.append(feed['url'].str.contains(url).any())
print(r)

[True, True, True, True, True, True]


In [108]:
r = []
for url in any_and_top:
    r.append(feed['url'].str.contains(url).any())
print(r)

[True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [109]:
any_and_top[9]

'https://www.gofundme.com/f/ceiling-hoist-lift-care-system-for-tucker-thigpe...'

In [110]:
from rapidfuzz import fuzz
from rapidfuzz import process

In [111]:
process.extractOne(any_and_top[9], feed['url'].tolist())

('https://www.gofundme.com/f/ceiling-hoist-lift-care-system-for-tucker-thigpen',
 99.33774834437087,
 23139)

In [112]:
any_and_top[9] = 'https://www.gofundme.com/f/ceiling-hoist-lift-care-system-for-tucker-thigpen'

In [113]:
m['url'] = m['url'].replace({'https://www.gofundme.com/f/ceiling-hoist-lift-care-system-for-tucker-thigpe':'https://www.gofundme.com/f/ceiling-hoist-lift-care-system-for-tucker-thigpen'})

In [114]:
top['url'] = top['url'].replace({'https://www.gofundme.com/f/ceiling-hoist-lift-care-system-for-tucker-thigpe':'https://www.gofundme.com/f/ceiling-hoist-lift-care-system-for-tucker-thigpen'})

In [115]:
r = []
for url in any_and_top:
    r.append(feed['url'].str.contains(url).any())
print(r)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [116]:
r = []
for url in any_only:
    r.append(feed['url'].str.contains(url).any())
print(r)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


### Make Survey

In [117]:
def MakeSurvey(l, feed, l_type, top, m):
    df = feed[feed['url'].isin(l)]
    results = []
    for index, row in df.iterrows():
        print(row['url'])
        if l_type == 'top_only':
            header = 'TOP [X], ANY [], BOTH []'
            dennis =  top[top['url'] == row['url']]['dennis'].tolist()[0]
            steven =  top[top['url'] == row['url']]['steven'].tolist()[0]
        if l_type == 'any_only':
            header = 'TOP [], ANY [X], BOTH []'
            dennis = m[m['url'] == row['url']]['dennis'].tolist()[0]
            steven =  m[m['url'] == row['url']]['steven'].tolist()[0]
        if l_type == 'any_and_top':
            header = 'TOP [], ANY [], BOTH [X]'
            dennis_top = top[top['url'] == row['url']]['dennis'].tolist()[0]
            dennis_any = m[m['url'] == row['url']]['dennis'].tolist()[0]
            dennis_notes = m[m['url'] == row['url']]['dennis'].tolist()[2]
            steven_top = top[top['url'] == row['url']]['steven'].tolist()[0]
            steven_any = m[m['url'] == row['url']]['steven'].tolist()[0]
            steven_notes = m[m['url'] == row['url']]['dennis'].tolist()[2]
        
        
        doc = '''
        [[AdvancedFormat]]

        [[Block:{0}]]
        

        [[Question:MC:MultipleAnswer:Vertical]]
        [[ID:{0}__base]]
        Disagreement: {1}
        <br />
        Dennis top: {2}
        <br />
        Dennis any: {3}
        <br />
        Dennis notes: {4}
        <br />
        <br />
        Steven top: {5}
        <br />
        Steven any: {6}
        <br />
        Steven notes: {7}
        <br />
        <br />
        
        
        {8}

        <br />
        <br />
        ------------------------------------------------
        <br />
        ------------------------------------------------
        <br />
        Please select all disease categories represented.
        [[Choices]]
        Cardiovascular diseases
        Endocrine diseases
        Gastrointestinal diseases
        Genitourinary disorders
        Infections
        Injuries and external causes
        Mental and substance use disorders
        Musculoskeletal diseases
        Neoplasms
        Nervous system diseases
        Nonspecific
        Other noncommunicable diseases
        Pregnancy and childbirth
        Respiratory diseases

        [[Question:MC:SingleAnswer:Vertical]]
        [[ID:{0}__top]]
        Please select the primary disease category.
        [[Choices]]
        Cardiovascular diseases
        Endocrine diseases
        Gastrointestinal diseases
        Genitourinary disorders
        Infections
        Injuries and external causes
        Mental and substance use disorders
        Musculoskeletal diseases
        Neoplasms
        Nervous system diseases
        Nonspecific
        Other noncommunicable diseases
        Pregnancy and childbirth
        Respiratory diseases

        [[Question:TE]]
        [[ID:{0}__NOTES]]
        Please make any notes below.

        [[PageBreak]]

        '''.format(row['url'],
                   header,
                   dennis_top,
                   dennis_any,
                   dennis_notes,
                   steven_top,
                   steven_any,
                   steven_notes,
                   row['fund_description'] 
                   )
        results.append(doc)
        
        
        doc = "\n".join(results)
        
        with open("Manual_review/{0}.txt".format(l_type), "w") as text_file:
            text_file.write(doc)
        
    return print('all done')

In [118]:
#old function format
#MakeSurvey(top_only, feed, 'top_only', top, m)

In [119]:
#old function format
#MakeSurvey(any_only, feed, 'any_only', top, m)

# Compile final reference set

### Import data from qualtrics review of discrepancies

In [173]:
resolved_top_only = pd.read_excel('Manual_Review/resolving_discrepancies/resolving_discrepancies_master.xlsx', sheet_name='top_only', header=None)
resolved_any_only = pd.read_excel('Manual_Review/resolving_discrepancies/resolving_discrepancies_master.xlsx', sheet_name='any_only', header=None)
resolved_any_and_top = pd.read_excel('Manual_Review/resolving_discrepancies/resolving_discrepancies_master.xlsx', sheet_name='any_and_top', header=None)

### Clean data

In [174]:
#rename columns
resolved_top_only.columns = ['qid','header','resp']
resolved_any_only.columns = ['qid','header','resp']
resolved_any_and_top.columns = ['qid','header','resp']

In [175]:
#extract url from header
resolved_top_only.loc[:,'url'] = [x.split(' - ')[0].split('__')[0] for x in resolved_top_only['header']]
resolved_any_only.loc[:,'url'] = [x.split(' - ')[0].split('__')[0] for x in resolved_any_only['header']]
resolved_any_and_top.loc[:,'url'] = [x.split(' - ')[0].split('__')[0] for x in resolved_any_and_top['header']]

In [176]:
print(resolved_top_only.shape)
print(resolved_any_only.shape)
print(resolved_any_and_top.shape)

(18, 4)
(132, 4)
(69, 4)


In [177]:
#add question category
resolved_top_only.loc[:,'cat'] = ['base','top','NOTES'] * int(18/3)
resolved_any_only.loc[:,'cat'] = ['base','top','NOTES'] * int(132/3)
resolved_any_and_top.loc[:,'cat'] = ['base','top','NOTES'] * int(69/3)

### Check that urls in each resolved category are unique

In [207]:
print(len(top_only))
print(len(any_only))
print(len(any_and_top))

print(resolved_top_only['url'].nunique())
print(resolved_any_only['url'].nunique())
print(resolved_any_and_top['url'].nunique())

6
44
23
6
44
23


In [215]:
print(sum([resolved_top_only['url'].str.contains(x).any() for x in resolved_any_only['url'].unique()]))
print(sum([resolved_top_only['url'].str.contains(x).any() for x in resolved_any_and_top['url'].unique()]))
print(sum([resolved_any_only['url'].str.contains(x).any() for x in resolved_any_and_top['url'].unique()]))

0
0
0


### Build new reference

In [236]:
#define function to build new reference set
def BuildReference(m, t, a, at):
    '''
    Inputs:
        m = as above, merged responses between dennis and steven
            (this will serve as the base, replace resolved discrepancies as needed)
        t = resolved_top_only
        a = resolved_any_only
        at = resolved_any_and_top
    '''
    
    #define results container
    r = []
    
    #loop through unique urls in m
    for url in m['url'].unique():
        
        #subset base data
        df = m[m['url'] == url]
        df = df.reset_index()

        if t['url'].str.contains(url).any():
            
            
            
            #susbet top data
            t_ = t[t['url'] == url]
            t_ = t_.reset_index()
            
            #get new value from resolved data
            new_value = t_[t_['url'] == url].iloc[1]['resp']
            
            #add resolved response to working base data
            df.at[1, 'steven'] = new_value
            
            #append results
            r.append(df)
            
        elif a['url'].str.contains(url).any():
            
            
            
            #susbet any data
            a_ = a[a['url'] == url]
            a_ = a_.reset_index()
            
            #get new value from resolved data
            new_value = a_[a_['url'] == url].iloc[0]['resp']
            
            #add resolved response to working base data
            df.at[0, 'steven'] = new_value
            
            #append results
            r.append(df)
            
        elif at['url'].str.contains(url).any():
            
            
            #susbet top data
            at_ = at[at['url'] == url]
            at_ = at_.reset_index()
            
            #get new value from resolved data
            new_value_any = at_[at_['url'] == url].iloc[0]['resp']
            new_value_top = at_[at_['url'] == url].iloc[1]['resp']
            
            #add resolved response to working base data
            df.at[0, 'steven'] = new_value_any
            df.at[1, 'steven'] = new_value_top
            
            #append results
            r.append(df)
            
        else:
            r.append(df)
    
    results = pd.concat(r)
    print('results shape: {0}'.format(results.shape))
    print('all done')
    return results

In [309]:
ref = BuildReference(m, resolved_top_only, resolved_any_only, resolved_any_and_top)

results shape: (600, 6)
all done


### Manually inspect some

In [310]:
resolved_top_only[:3]

,qid,header,resp,url,cat
0,QID20,https://www.gofundme.com/f/help-me-see-your-wo...,,https://www.gofundme.com/f/help-me-see-your-world,base
1,QID21,https://www.gofundme.com/f/help-me-see-your-wo...,Nervous system diseases,https://www.gofundme.com/f/help-me-see-your-world,top
2,QID22_TEXT,https://www.gofundme.com/f/help-me-see-your-wo...,interpreting primary medical condition as the ...,https://www.gofundme.com/f/help-me-see-your-world,NOTES


In [311]:
ref[ref['url'] == 'https://www.gofundme.com/f/help-me-see-your-world']

,index,url,qcat,dennis,steven,round
0,35,https://www.gofundme.com/f/help-me-see-your-world,base,"Nervous system diseases,Other noncommunicable ...","Nervous system diseases,Other noncommunicable ...",3
1,36,https://www.gofundme.com/f/help-me-see-your-world,top,Other noncommunicable diseases,Nervous system diseases,3
2,37,https://www.gofundme.com/f/help-me-see-your-world,NOTES,NaN,"H47619\tCortical blindness, unspecified side o...",3


In [312]:
resolved_any_only.iloc[0]['resp']

'Infections,Injuries and external causes,Mental and substance use disorders,Nonspecific'

In [313]:
ref[ref['url'] == resolved_any_only.iloc[0]['url']].iloc[0]['steven']

'Infections,Injuries and external causes,Mental and substance use disorders,Nonspecific'

In [314]:
resolved_any_and_top[3:6]

,qid,header,resp,url,cat
3,QID5,https://www.gofundme.com/f/xny684__base - Disa...,"Endocrine diseases,Nonspecific",https://www.gofundme.com/f/xny684,base
4,QID6,https://www.gofundme.com/f/xny684__top - Pleas...,Endocrine diseases,https://www.gofundme.com/f/xny684,top
5,QID7_TEXT,https://www.gofundme.com/f/xny684__NOTES - Ple...,NaN,https://www.gofundme.com/f/xny684,NOTES


In [315]:
ref[ref['url'] == 'https://www.gofundme.com/f/xny684']

,index,url,qcat,dennis,steven,round
0,87,https://www.gofundme.com/f/xny684,base,Endocrine diseases,"Endocrine diseases,Nonspecific",6
1,88,https://www.gofundme.com/f/xny684,top,Endocrine diseases,Endocrine diseases,6
2,89,https://www.gofundme.com/f/xny684,NOTES,NaN,NaN,6


### Clean new reference set

In [316]:
#subset columns
ref = ref[['url','qcat','steven']]
ref = ref.rename(columns={'steven':'label'})

In [317]:
#check that urls are same as feed (some are abbreviated from qualtrics)
urls_to_replace = ref[~ref['url'].isin(feed['url'])]['url'].unique().tolist()
urls_to_replace

['https://www.gofundme.com/f/evp227-fernando-diaz',
 'https://www.gofundme.com/f/esperanza-para-mi-viejito-transplante-de-pulmon_...',
 'https://www.gofundme.com/f/ceiling-hoist-lift-care-system-for-tucker-thigpe...',
 'https://www.gofundme.com/f/lets-do-our-part-to-help-the-front-line-workers_...',
 'https://www.gofundme.com/f/help-noah-fight-stage-4-neuroendocrine-carcinoma...',
 'https://www.gofundme.com/f/get-mike-home-via-a-visit-to-his-mom-and-the-wat...']

In [318]:
#'https://www.gofundme.com/f/evp227-fernando-diaz' is probably before filtering out spanish campaigns but it has english and was labeled
ref[ref['url'] == 'https://www.gofundme.com/f/evp227-fernando-diaz']

,url,qcat,label
0,https://www.gofundme.com/f/evp227-fernando-diaz,base,Neoplasms
1,https://www.gofundme.com/f/evp227-fernando-diaz,top,Neoplasms
2,https://www.gofundme.com/f/evp227-fernando-diaz,NOTES,NaN


In [319]:
new_urls = []
for url in urls_to_replace[1:]:
    new_urls.append(process.extractOne(url, feed['url'].tolist())[0])
ref = ref.replace(urls_to_replace[1:], new_urls)

In [320]:
#checking that all were replaced
ref[~ref['url'].isin(feed['url'])]['url'].unique().tolist()

['https://www.gofundme.com/f/evp227-fernando-diaz']

In [321]:
#subset data by question category
ref_any = ref[ref['qcat'] == 'base']
ref_top = ref[ref['qcat'] == 'top']

In [322]:
print(ref_any.shape)
print(ref_top.shape)

(200, 3)
(200, 3)


In [323]:
ref_any.columns

Index(['url', 'qcat', 'label'], dtype='object')

In [324]:
def comma2lofd(df):
    '''convert cat column from comma delimited to list of dictionaries with 1 or 0 for each category'''
    
    disease_categories = [
        'Cardiovascular diseases',
        'Endocrine diseases',
        'Gastrointestinal diseases',
        'Genitourinary disorders',
        'Infections',
        'Injuries and external causes',
        'Mental and substance use disorders',
        'Musculoskeletal diseases',
        'Neoplasms',
        'Nervous system diseases',
        'Nonspecific',
        'Other noncommunicable diseases',
        'Pregnancy and childbirth',
        'Respiratory diseases'
    ]
    
    r = []
    
    for index,row in df.iterrows():
        lofd = []
        as_list = row['label'].split(',')
        for disease in disease_categories:
            if disease in as_list:
                lofd.append({disease:1})
            else:
                lofd.append({disease:0})
        r.append(lofd)
    return r

In [332]:
ref_any['lofd'] = comma2lofd(ref_any)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [334]:
#convert to long format
long = ref_any.explode('lofd')

In [335]:
#extract key, value from dictionary
long.insert(loc=2, column='cat', value=[list(x.items())[0][0] for x in long['lofd']])
long.insert(loc=3, column='count', value=[list(x.items())[0][1] for x in long['lofd']])

In [337]:
long = long[['url','cat','count']]

In [348]:
#create pivot table
pivot = long.pivot_table(index=['url'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)

In [350]:
pivot.columns.name = ''
pivot = pivot.reset_index()

### Export data

In [355]:
ref_top.to_csv('Manual_Review/reference_set/reference_set_top.csv', index=False)
long.to_csv('Manual_Review/reference_set/reference_set_any_long.csv', index=False)
pivot.to_csv('Manual_Review/reference_set/reference_set_any_pivot.csv', index=False)